In [6]:
import math
import tensorflow as tf
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import os
from tensorflow.contrib import rnn


tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    mpl.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = np.float32)

xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))



seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1001
epsilon = 1
e = math.exp(epsilon)
q = (1/(e+1))

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)

X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기
with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_mean(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)


with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    #multi_cell2 =  tf.contrib.rnn.MultiRNNCell([cell2]*2)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)




#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))


x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])*2
x2 = x1+0.33*2
x3 = x2+0.33*2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[nstep: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})
    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    rmse_val_with_noise = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict_with_noise})
    print("RMSE: {} \nRMSE_with_noise{} ".format(rmse_val,rmse_val_with_noise))

[step: 0] loss: 0.2825731635093689
[nstep: 0] loss: 0.2595043480396271
[step: 1] loss: 0.1896384209394455
[nstep: 1] loss: 0.15791241824626923
[step: 2] loss: 0.13416717946529388
[nstep: 2] loss: 0.10182338207960129
[step: 3] loss: 0.09712933748960495
[nstep: 3] loss: 0.06897313892841339
[step: 4] loss: 0.07193728536367416
[nstep: 4] loss: 0.04919050633907318
[step: 5] loss: 0.0542481429874897
[nstep: 5] loss: 0.038430724292993546
[step: 6] loss: 0.042936116456985474
[nstep: 6] loss: 0.03298943489789963
[step: 7] loss: 0.03701147809624672
[nstep: 7] loss: 0.03087751939892769
[step: 8] loss: 0.03462768718600273
[nstep: 8] loss: 0.02905426360666752
[step: 9] loss: 0.033082686364650726
[nstep: 9] loss: 0.027896417304873466
[step: 10] loss: 0.0323486402630806
[nstep: 10] loss: 0.02676728367805481
[step: 11] loss: 0.03149361535906792
[nstep: 11] loss: 0.025134265422821045
[step: 12] loss: 0.030025651678442955
[nstep: 12] loss: 0.02340037189424038
[step: 13] loss: 0.028489528223872185
[nstep

[step: 108] loss: 0.0027127184439450502
[nstep: 108] loss: 0.0076850526966154575
[step: 109] loss: 0.002654195064678788
[nstep: 109] loss: 0.0076999724842607975
[step: 110] loss: 0.0026469624135643244
[nstep: 110] loss: 0.007661112118512392
[step: 111] loss: 0.0026323869824409485
[nstep: 111] loss: 0.007674681954085827
[step: 112] loss: 0.0026303455233573914
[nstep: 112] loss: 0.007649548351764679
[step: 113] loss: 0.002628446090966463
[nstep: 113] loss: 0.007656483445316553
[step: 114] loss: 0.00262809288688004
[nstep: 114] loss: 0.007627964951097965
[step: 115] loss: 0.0026157370302826166
[nstep: 115] loss: 0.0076329829171299934
[step: 116] loss: 0.0025994819588959217
[nstep: 116] loss: 0.00762520544230938
[step: 117] loss: 0.00257113273255527
[nstep: 117] loss: 0.007610914763063192
[step: 118] loss: 0.0025726098101586103
[nstep: 118] loss: 0.007607107982039452
[step: 119] loss: 0.002539076842367649
[nstep: 119] loss: 0.007594940718263388
[step: 120] loss: 0.0025404999032616615
[nste

[step: 213] loss: 0.0019836854189634323
[nstep: 213] loss: 0.007242561783641577
[step: 214] loss: 0.001981691922992468
[nstep: 214] loss: 0.007240201812237501
[step: 215] loss: 0.001968687167391181
[nstep: 215] loss: 0.007238265126943588
[step: 216] loss: 0.002051816787570715
[nstep: 216] loss: 0.0072363149374723434
[step: 217] loss: 0.0019484504591673613
[nstep: 217] loss: 0.007234096061438322
[step: 218] loss: 0.001963501563295722
[nstep: 218] loss: 0.007231646683067083
[step: 219] loss: 0.0019207514123991132
[nstep: 219] loss: 0.007229014299809933
[step: 220] loss: 0.001914613414555788
[nstep: 220] loss: 0.007226303219795227
[step: 221] loss: 0.0019510386046022177
[nstep: 221] loss: 0.007223615422844887
[step: 222] loss: 0.0019164454424753785
[nstep: 222] loss: 0.007221124600619078
[step: 223] loss: 0.0019730604253709316
[nstep: 223] loss: 0.0072187138721346855
[step: 224] loss: 0.0019189994782209396
[nstep: 224] loss: 0.007216412108391523
[step: 225] loss: 0.0019402352627366781
[ns

[step: 318] loss: 0.0017631863011047244
[nstep: 318] loss: 0.0071438634768128395
[step: 319] loss: 0.0016686898889020085
[nstep: 319] loss: 0.007079340983182192
[step: 320] loss: 0.0016005783108994365
[nstep: 320] loss: 0.007052287925034761
[step: 321] loss: 0.001659587025642395
[nstep: 321] loss: 0.0070822699926793575
[step: 322] loss: 0.0017652657115831971
[nstep: 322] loss: 0.007108366582542658
[step: 323] loss: 0.0017766357632353902
[nstep: 323] loss: 0.007093213032931089
[step: 324] loss: 0.0017061992548406124
[nstep: 324] loss: 0.007056375499814749
[step: 325] loss: 0.001616513472981751
[nstep: 325] loss: 0.0070488243363797665
[step: 326] loss: 0.0015997713198885322
[nstep: 326] loss: 0.007068807724863291
[step: 327] loss: 0.0016164594562724233
[nstep: 327] loss: 0.007078174035996199
[step: 328] loss: 0.0016524572856724262
[nstep: 328] loss: 0.007064543664455414
[step: 329] loss: 0.001649287878535688
[nstep: 329] loss: 0.007044881582260132
[step: 330] loss: 0.001623644377104938
[

[step: 423] loss: 0.0013550280127674341
[nstep: 423] loss: 0.007049852516502142
[step: 424] loss: 0.001357631292194128
[nstep: 424] loss: 0.0069872550666332245
[step: 425] loss: 0.001353312167339027
[nstep: 425] loss: 0.006926701404154301
[step: 426] loss: 0.0013494818704202771
[nstep: 426] loss: 0.006939860992133617
[step: 427] loss: 0.0013511490542441607
[nstep: 427] loss: 0.006989746820181608
[step: 428] loss: 0.001339693320915103
[nstep: 428] loss: 0.006996465846896172
[step: 429] loss: 0.0013556394260376692
[nstep: 429] loss: 0.006957497913390398
[step: 430] loss: 0.0013435827568173409
[nstep: 430] loss: 0.00691911531612277
[step: 431] loss: 0.0013582557439804077
[nstep: 431] loss: 0.006925318390130997
[step: 432] loss: 0.0013678019167855382
[nstep: 432] loss: 0.0069530983455479145
[step: 433] loss: 0.001406405819579959
[nstep: 433] loss: 0.006958415266126394
[step: 434] loss: 0.0014293543063104153
[nstep: 434] loss: 0.006937058176845312
[step: 435] loss: 0.0015131495893001556
[ns

[nstep: 527] loss: 0.007038378156721592
[step: 528] loss: 0.0011375516187399626
[nstep: 528] loss: 0.007172577083110809
[step: 529] loss: 0.0013355358969420195
[nstep: 529] loss: 0.007203278597444296
[step: 530] loss: 0.0014529043110087514
[nstep: 530] loss: 0.007160307373851538
[step: 531] loss: 0.001263206941075623
[nstep: 531] loss: 0.0069892145693302155
[step: 532] loss: 0.0011603140737861395
[nstep: 532] loss: 0.006893383339047432
[step: 533] loss: 0.0013133693719282746
[nstep: 533] loss: 0.006923542357981205
[step: 534] loss: 0.0012322135735303164
[nstep: 534] loss: 0.006975880824029446
[step: 535] loss: 0.0011557573452591896
[nstep: 535] loss: 0.006984974257647991
[step: 536] loss: 0.0012798183597624302
[nstep: 536] loss: 0.006920071318745613
[step: 537] loss: 0.0011688111117109656
[nstep: 537] loss: 0.006886479444801807
[step: 538] loss: 0.001149671501480043
[nstep: 538] loss: 0.0068789585493505
[step: 539] loss: 0.0012464530300348997
[nstep: 539] loss: 0.006893464829772711
[st

[step: 632] loss: 0.0009945810306817293
[nstep: 632] loss: 0.006734365131705999
[step: 633] loss: 0.00098249816801399
[nstep: 633] loss: 0.006731532979756594
[step: 634] loss: 0.0009769430616870522
[nstep: 634] loss: 0.006748104467988014
[step: 635] loss: 0.0009835987584665418
[nstep: 635] loss: 0.006759243551641703
[step: 636] loss: 0.0009709590231068432
[nstep: 636] loss: 0.006750766653567553
[step: 637] loss: 0.000972562818787992
[nstep: 637] loss: 0.006733338814228773
[step: 638] loss: 0.0009730241727083921
[nstep: 638] loss: 0.006724617909640074
[step: 639] loss: 0.0009658714989200234
[nstep: 639] loss: 0.006729770917445421
[step: 640] loss: 0.0009648678242228925
[nstep: 640] loss: 0.006739625707268715
[step: 641] loss: 0.0009609140106476843
[nstep: 641] loss: 0.006741310469806194
[step: 642] loss: 0.000964642851613462
[nstep: 642] loss: 0.006733651272952557
[step: 643] loss: 0.0009580575278960168
[nstep: 643] loss: 0.0067234947346150875
[step: 644] loss: 0.0009540297905914485
[ns

[step: 737] loss: 0.0008891624747775495
[nstep: 737] loss: 0.006655190140008926
[step: 738] loss: 0.0008668277296237648
[nstep: 738] loss: 0.006650969386100769
[step: 739] loss: 0.0008822676027193666
[nstep: 739] loss: 0.006648290902376175
[step: 740] loss: 0.0008811270818114281
[nstep: 740] loss: 0.006648400332778692
[step: 741] loss: 0.0008717502350918949
[nstep: 741] loss: 0.00664916168898344
[step: 742] loss: 0.0008651360985822976
[nstep: 742] loss: 0.006647935137152672
[step: 743] loss: 0.0008667154470458627
[nstep: 743] loss: 0.006645668298006058
[step: 744] loss: 0.0008757178438827395
[nstep: 744] loss: 0.006644308101385832
[step: 745] loss: 0.0008606563787907362
[nstep: 745] loss: 0.006644456181675196
[step: 746] loss: 0.0008550375932827592
[nstep: 746] loss: 0.006644811015576124
[step: 747] loss: 0.000863206631038338
[nstep: 747] loss: 0.0066437250934541225
[step: 748] loss: 0.0008585715549997985
[nstep: 748] loss: 0.006641803774982691
[step: 749] loss: 0.0008567097247578204
[

[step: 840] loss: 0.0008277601446025074
[nstep: 840] loss: 0.006579011678695679
[step: 841] loss: 0.0008159168064594269
[nstep: 841] loss: 0.006578449625521898
[step: 842] loss: 0.0007982272654771805
[nstep: 842] loss: 0.006578155793249607
[step: 843] loss: 0.0007882168283686042
[nstep: 843] loss: 0.006577820982784033
[step: 844] loss: 0.0007916198228485882
[nstep: 844] loss: 0.006577370688319206
[step: 845] loss: 0.0007898719632066786
[nstep: 845] loss: 0.006576718296855688
[step: 846] loss: 0.0007841627229936421
[nstep: 846] loss: 0.006575870793312788
[step: 847] loss: 0.0007690333877690136
[nstep: 847] loss: 0.006574958562850952
[step: 848] loss: 0.0007608581800013781
[nstep: 848] loss: 0.006574012339115143
[step: 849] loss: 0.0007613966008648276
[nstep: 849] loss: 0.006573163904249668
[step: 850] loss: 0.0007663427968509495
[nstep: 850] loss: 0.00657236110419035
[step: 851] loss: 0.0007730491342954338
[nstep: 851] loss: 0.00657167611643672
[step: 852] loss: 0.0007765564369037747
[n

[step: 945] loss: 0.0007338337600231171
[nstep: 945] loss: 0.006519890855997801
[step: 946] loss: 0.0007214095094241202
[nstep: 946] loss: 0.006519325543195009
[step: 947] loss: 0.0007095055771060288
[nstep: 947] loss: 0.006518935319036245
[step: 948] loss: 0.0007117530331015587
[nstep: 948] loss: 0.006518423557281494
[step: 949] loss: 0.0007225774461403489
[nstep: 949] loss: 0.0065177553333342075
[step: 950] loss: 0.0007257873076014221
[nstep: 950] loss: 0.006517181172966957
[step: 951] loss: 0.0007159580709412694
[nstep: 951] loss: 0.006516674067825079
[step: 952] loss: 0.0007041627541184425
[nstep: 952] loss: 0.006516084540635347
[step: 953] loss: 0.0007037617033347487
[nstep: 953] loss: 0.006515459157526493
[step: 954] loss: 0.0007108788704499602
[nstep: 954] loss: 0.006514930631965399
[step: 955] loss: 0.000719198549631983
[nstep: 955] loss: 0.006514470558613539
[step: 956] loss: 0.0007233831565827131
[nstep: 956] loss: 0.006513969972729683
[step: 957] loss: 0.0007367093930952251


# Correlation

In [2]:
def correlation(data1,data2):
    

    data1_mean = np.mean(data1)
    data2_mean = np.mean(data2)

    X_1 = data1-data1_mean
    Y_1 = data2-data2_mean
    #분자
    a_1 = np.sum(np.multiply(X_1,Y_1))

    #분모
    b_1 = np.sqrt(np.sum(np.square(X_1)))
    c_1 = np.sqrt(np.sum(np.square(Y_1)))

    #상관계수
    r_1 = (a_1/(b_1*c_1))

    print("correlation(real&predict) :{}".format(r_1))


correlation(real&predict) :0.9838788016409673
correlation(real&noise) :0.9408963627942616


In [12]:
tnp = np.corrcoef(testY,test_predict)
tnn = np.corrcoef(testY,test_predict_with_noise)
print(tnp,tnn)

[[1.         0.9878891  0.96181054 ... 0.57012402 0.55135712 0.70255796]
 [0.9878891  1.         0.98099501 ... 0.5611616  0.53883772 0.68989686]
 [0.96181054 0.98099501 1.         ... 0.60776359 0.58202756 0.72302576]
 ...
 [0.57012402 0.5611616  0.60776359 ... 1.         0.98879793 0.97421917]
 [0.55135712 0.53883772 0.58202756 ... 0.98879793 1.         0.95518998]
 [0.70255796 0.68989686 0.72302576 ... 0.97421917 0.95518998 1.        ]] [[1.         0.9878891  0.96181054 ... 0.44971721 0.59663495 0.60433858]
 [0.9878891  1.         0.98099501 ... 0.45095317 0.57352385 0.58368285]
 [0.96181054 0.98099501 1.         ... 0.49497364 0.59088409 0.60590495]
 ...
 [0.44971721 0.45095317 0.49497364 ... 1.         0.93935503 0.94930187]
 [0.59663495 0.57352385 0.59088409 ... 0.93935503 1.         0.97327264]
 [0.60433858 0.58368285 0.60590495 ... 0.94930187 0.97327264 1.        ]]


In [3]:
index_y = np.array(range(0,len(testY)))
index_x = np.array(range(0,25))

# 3D Graph

In [ ]:
index_y = np.array(range(0,len(testY)))
index_x = np.array(range(0,25))
index_x_grid, index_y_grid = np.meshgrid(index_x,index_y)
index_x_3d,index_y_3d = index_x_grid.ravel(),index_y_grid.ravel()

fig = plt.figure()
ax1 = fig.add_subplot(111,projection='3d')
wire = ax1.plot_wireframe(index_x_grid*2,index_y_grid*2,testY,color='r',linewidth=0.2)
wire = ax1.plot_wireframe(index_x_grid*2,index_y_grid*2,test_predict,color='g',linewidth=0.2)


plt.show()

In [5]:
index_y = np.array(range(0,len(testY)))
index_x = np.array(range(0,25))
index_x_grid, index_y_grid = np.meshgrid(index_x,index_y)
index_x_3d,index_y_3d = index_x_grid.ravel(),index_y_grid.ravel()


fig = plt.figure()
ax2 = fig.add_subplot(111,projection='3d')
wire = ax2.plot_wireframe(index_x_grid*2,index_y_grid*2,testY,color='r',linewidth=0.2)
wire = ax2.plot_wireframe(index_x_grid*2,index_y_grid*2,test_predict_with_noise,color='b',linewidth=0.2)
plt.show()


KeyboardInterrupt: 

# Euclidean Distance & Dynamic Time Warping

In [3]:
#Euclidean Distance
ED = abs(testY[:,0]-test_predict[:,0])
print(ED,"\n",np.mean(ED),"\n",sum(ED))

[0.02466743 0.03248346 0.08497622 ... 0.01174273 0.03616501 0.05264915] 
 0.03160617722748938 
 38.05383738189723


In [4]:
rmp = testY[:,0] - test_predict[:,0]
rmp_2 = np.square(rmp)
ED_2 = np.sqrt(sum(rmp_2))
print(ED_2)

1.5783497323946785


In [4]:
from dtw import dtw

In [5]:
x = np.array(testY[:,0]).reshape(-1,1)
y = np.array(test_predict[:,0]).reshape(-1,1)

EN = lambda x, y: np.abs(x - y)

d, cost_matrix, acc_cost_matrix, path = dtw(x, y, dist=EN)
print(d)

0.01949119639398486


In [6]:
cost_matrix

array([[0.06399202, 0.05678671, 0.16861615, ..., 0.04548428, 0.11664701,
        0.03301746],
       [0.05242814, 0.04522283, 0.15705228, ..., 0.05704816, 0.12821089,
        0.04458134],
       [0.04503882, 0.05224413, 0.05958532, ..., 0.15451512, 0.22567785,
        0.1420483 ],
       ...,
       [0.15255663, 0.14535132, 0.25718077, ..., 0.04308033, 0.0280824 ,
        0.05554715],
       [0.20762271, 0.2004174 , 0.31224684, ..., 0.09814641, 0.02698368,
        0.11061323],
       [0.20248321, 0.1952779 , 0.30710734, ..., 0.09300691, 0.02184418,
        0.10547373]])

In [7]:
acc_cost_matrix[-1,-1]

3.7033273148571233

In [8]:
d

0.01949119639398486

In [27]:
path

(array([ 0,  1,  2,  3,  4,  5,  6,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 19, 20, 21, 22, 22, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 32, 33, 34, 35, 36, 37, 38, 38, 38, 38, 38, 38, 38, 39, 40, 41,
        42, 43, 44, 45, 46, 47, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 56,
        56, 56, 56, 56, 57, 58, 59, 60, 61, 62, 63, 63, 64, 64, 65, 65, 65,
        66, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
        82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 92, 92, 92, 92, 93, 94,
        94, 94]),
 array([ 0,  1,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 17, 18, 19, 20, 21, 22, 23, 24, 24, 25, 26, 27, 28, 29, 30,
        31, 32, 32, 32, 32, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
        43, 43, 44, 45, 45, 46, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 53,
        54, 55, 56, 57, 58, 58, 59, 59, 60, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 70, 71, 72, 73, 74, 75, 76, 77, 78, 78, 78, 79, 79

In [ ]:
def DTW(A, B, window=sys.maxint, d=lambda x, y: abs(x - y)):
# 비용 행렬 초기화 
    A, B = np.array(A), np.array(B)
    M, N = len(A), len(B)
    cost = sys.maxint * np.ones((M, N))

# 첫번째 로우,컬럼 채우기
    cost[0, 0] = d(A[0], B[0])
    for i in range(1, M):
        cost[i, 0] = cost[i - 1, 0] + d(A[i], B[0])

    for j in range(1, N):
        cost[0, j] = cost[0, j - 1] + d(A[0], B[j])
# 나머지 행렬 채우기 
    for i in range(1, M):
        for j in range(max(1, i - window), min(N, i + window)):
            choices = cost[i - 1, j - 1], cost[i, j - 1], cost[i - 1, j]
            cost[i, j] = min(choices) + d(A[i], B[j])

# 최적 경로 구하기 
    n, m = N - 1, M - 1
    path = []

    while (m, n) != (0, 0):
        path.append((m, n))
        m, n = min((m - 1, n), (m, n - 1), (m - 1, n - 1), key=lambda x: cost[x[0], x[1]])

    path.append((0, 0))
    return cost[-1, -1], path



1. Seoul_Jongno-gu,
2. Seoul_Jung-gu,
3. Seoul_Yongsan-gu,
4. Seoul_Seongdong-gu,
5. Seoul_Gwangjin-gu,
6. Seoul_Dongdaemun-gu, 
7. Seoul_Jungnang-gu,
8. Seoul_Seongbuk-gu,
9. Seoul_Gangbuk-gu,
10. Seoul_Dobong-gu,
11. Seoul_Nowon-gu,
12. Seoul_Eunpyeong-gu,
13. Seoul_Seodaemun-gu,
14. Seoul_Mapo-gu,
15. Seoul_Yangcheon-gu,
16. Seoul_Gangseo-gu,
17. Seoul_Guro-gu,
18. Seoul_Geumcheon-gu,
19. Seoul_Yeongdeungpo-gu,
20. Seoul_Dongjak-gu,
21. Seoul_Gwanak-gu,
22. Seoul_Seocho-gu,
23. Seoul_Gangnam-gu,
24. Seoul_Songpa-gu,
25. Seoul_Gangdong-gu,

In [48]:
plt.plot(index_y,testY[:,11],label = 'real',color ='r')
plt.plot(index_y,test_predict[:,11],label = 'predict',color ='b')
plt.plot(index_y,test_predict_with_noise[:,11],label = 'noise',color ='g')
plt.title('Eunpyeong-gu')
plt.legend()
plt.show()

KeyboardInterrupt: 